In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
cd My Drive

In [0]:
!gsutil cp gs://coronawhy/NLPDatasets/v6_preprocessed/v6_text.zip .
!unzip v6_text.zip

Copying gs://coronawhy/NLPDatasets/v6_preprocessed/v6_text.zip...
\ [1 files][  1.0 GiB/  1.0 GiB]   44.9 MiB/s                                   
Operation completed over 1 objects/1.0 GiB.                                      


In [0]:
import numpy as np 
import pandas as pd 
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import xml.etree.ElementTree as ET

In [0]:
df = pd.read_pickle("/content/gdrive/My Drive/v6_text/v6_text_0.pkl", compression = "gzip")

In [0]:
#Let me load my clinical trials and get the unique IDs. This is the publicly available WHO clinical trials dataset
trials = pd.read_csv("/content/gdrive/My Drive/COVID19.csv")
trials = trials['TrialID']
trials = pd.unique(trials)
import re
def trial_cleaner(data):

  data = re.sub(r'\W+', '', data).lower()
  return data
trials = trials.apply(trial_cleaner)
trials = list(trials)

In [0]:
#Global set of paper Ids
pid = dict()
for trial in trials:
  pid[trial] = list()

In [0]:
#Master Code
root_dir = "v6_text"
x = os.walk("v6_text",topdown=False)
for root,dir,file in x:
  for name in file:
    file_name = os.path.join(root_dir,name)
    df = pd.read_pickle(file_name, compression = "gzip")
    df = df.dropna(subset=['sentence'])
    df2 = df.groupby('paper_id')
    grouped_lists = df2['sentence'].agg(lambda x: " ".join(x))
    grouped_lists = grouped_lists.reset_index(name="sentence")
    grouped_lists['sentence']=grouped_lists['sentence'].str.lower()
    for k in pid.keys():
      for item in grouped_lists[grouped_lists['sentence'].str.contains(k)]['paper_id']:
        pid[k].append(item)
    

In [0]:
import json
json.dump(pid,open("mappings.json", 'w'))

In [0]:
import json
with open('mappings.json', 'r') as fp:
    data = json.load(fp)

In [0]:
matches = pd.DataFrame([(key, var) for (key, L) in data.items() for var in L], columns=['key', 'paper_id'])


In [0]:
matches

,key,paper_id
0,nct04246242,e3194f34badda0d3da79d2c00ec844bc24269e73
1,nct04252885,f6b9fd7dbed6516888eca71004caaa62aa0132d4
2,chictr2000029935,4b2b9841f4d7064851b89540f55df916f279a82b
3,nct04261907,58be092086c74c58e9067121a6ba4836468e7ec3
4,chictr2000029939,4b2b9841f4d7064851b89540f55df916f279a82b
...,...,...
78,nct04303507,eff6ce20122149117020cda90ad17f7cff0e9fd4
79,nct04283461,51cf515868c3f98cacf6c8fe12625714b3483331
80,nct04280705,05d47dd5b46f86428de058db4ecc8bca76a9ad16
81,nct04280705,eba1da4b90ee7cff2811391e12e773baa5cd6c2e


In [0]:
linked = trials.merge(matches, left_on='cleaned_trial_id', right_on='key',how='left')

In [0]:
!pip install geopy

In [0]:
from geopy.geocoders import Nominatim, GeoNames, DataBC, ArcGIS, TomTom, Bing, GoogleV3
geolocator = Bing(api_key="API_KEY")


In [0]:
from geopy.extra.rate_limiter import RateLimiter
def locparse(s):
    try:
      geoc = RateLimiter(geolocator.geocode, min_delay_seconds=2)
      return geoc(s).raw['address']['countryRegion']

    except:
      return np.nan


In [0]:
linked['parsed_country'] = linked.apply(lambda x: locparse(x['Primary sponsor']) if not pd.notnull(x['Countries']) else x['Countries'], axis=1)

In [0]:
linked_compressed = linked.loc[:, ['TrialID', 'paper_id','parsed_country']]

In [0]:
linked_compressed.to_csv("ClinicalTrialsLinked.csv")